In [ ]:
import pandas as pd
import xgboost as xgb

In [ ]:
df = pd.read_csv('../data/newborn_train.csv')
df_test = pd.read_csv('../data/newborn_test.csv')

In [ ]:
best_params = {
    "n_estimators": [10, 25, 50, 100, 250],
    "eta": [0.1, 0.15, 0.2],
    "max_depth": [3, 5, 7],
    "colsample_bytree": [0.8, 0.9],
    "gamma": [0, 1, 3, 5],
    "eval_metric": ["mape"],
    "objective": ["reg:absoluteerror", "reg:squarederror"],
    "booster": ["gbtree"],
    "tree_method": ["gpu_hist"],
    "gpu_id": [0],
}

In [ ]:
model = xgb.XGBRegressor(**best_params)

In [ ]:
y = df.newborn_weight
df = df.drop(columns=['newborn_weight'])

In [ ]:
model.fit(df, y)

In [ ]:
preds = model.predict(df_test)

In [ ]:
preds.to_csv('../data/newborn_test_preds.csv')